In [ ]:
# Check GPU
import torch
print('Torch:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU name:', torch.cuda.get_device_name(0))
else:
    print('Enable GPU in Runtime > Change runtime type')


In [ ]:
# Option A: Mount Google Drive and use existing project in Drive (edit the path)
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)

PROJECT_DIR = '/content/drive/MyDrive/your_project_path/genai_A1'  # <-- EDIT THIS
Q2_DIR = os.path.join(PROJECT_DIR, 'Q2')

%cd $Q2_DIR
os.environ['PYTHONPATH'] = Q2_DIR
!pwd && ls -la


In [ ]:
# Option B: Clone from GitHub (replace with your repo URL)
import os

%cd /content
!git clone https://github.com/your/repo.git genai_A1
%cd /content/genai_A1/Q2
os.environ['PYTHONPATH'] = '/content/genai_A1/Q2'
!pwd && ls -la


In [ ]:
# Option C: Upload a ZIP and unzip to /content
from google.colab import files
import os

uploaded = files.upload()  # upload your project zip
zip_name = list(uploaded.keys())[0]
!unzip -o "$zip_name" -d /content
%cd /content/genai_A1/Q2
os.environ['PYTHONPATH'] = '/content/genai_A1/Q2'
!pwd && ls -la


In [ ]:
# Install dependencies
%pip install -U pip
%pip install -r /content/genai_A1/requirements.txt

# Optional: Speed up HF datasets cache
import os
os.environ['HF_DATASETS_CACHE'] = '/content/hf_cache'
!mkdir -p /content/hf_cache


In [ ]:
# Ensure we are in Q2 dir and outputs exists
import os, pathlib
Q2_DIR = os.getcwd()
print('Working dir:', Q2_DIR)
os.makedirs('outputs', exist_ok=True)
!ls -la


In [ ]:
# ===============================================
# TASK 1: Dataset Loading and Preprocessing
# ===============================================
print("🚀 TASK 1: Loading and Preprocessing Shakespeare Dataset...")

# Test dataset loading from HuggingFace
!python -c "from src.dataset import ShakespeareDataset; ds = ShakespeareDataset(split='train', seq_len=50); print(f'✅ Dataset loaded! Vocab size: {ds.vocab_size}, Sequences: {len(ds)}')"

# Test model creation
!python -c "from src.model import ShakespeareRNN; model = ShakespeareRNN(vocab_size=100, embedding_dim=128, hidden_size=256); print(f'✅ RNN model created with {sum(p.numel() for p in model.parameters()):,} parameters')"

print("✅ TASK 1 COMPLETE: Dataset loaded and preprocessed!")


In [ ]:
# ===============================================
# TASK 2: RNN Model Implementation and Training
# ===============================================
print("🧠 TASK 2: Training RNN Model with Custom Embeddings...")

# Train baseline RNN model (balanced epochs for good performance)
!python -m src.train --epochs 20 --batch_size 64 --seq_len 100 --lr 0.001 --embedding_dim 128 --hidden_size 256 --num_layers 2 --rnn_type LSTM --outdir outputs

# Display training curves
from IPython.display import Image, display
print("📈 Training and Validation Curves:")
display(Image('outputs/training_curves.png'))

print("✅ TASK 2 COMPLETE: RNN model trained with custom embeddings!")


In [ ]:
# ===============================================
# TASK 3: Text Generation with Seed Phrases
# ===============================================
print("📝 TASK 3: Generating Text with Seed Phrases...")

# Generate text with various seed phrases
seed_phrases = [
    "To be or not to",
    "Once upon a time",
    "The quick brown fox",
    "All the world's a",
    "In fair Verona"
]

print("🎭 Generating Shakespeare-style text with seed phrases:")
for i, seed in enumerate(seed_phrases, 1):
    print(f"\n{i}. Seed: '{seed}'")
    !python -c "
from src.model import ShakespeareRNN
from src.utils import generate_text
import torch
import json

# Load model and generate
try:
    checkpoint = torch.load('outputs/best_model.pt', map_location='cpu')
    vocab_info = checkpoint['vocab_info']
    model_info = checkpoint['model_info']
    
    model = ShakespeareRNN(
        vocab_size=model_info['vocab_size'],
        embedding_dim=model_info['embedding_dim'],
        hidden_size=model_info['hidden_size'],
        num_layers=model_info['num_layers'],
        rnn_type=model_info['rnn_type']
    )
    model.load_state_dict(checkpoint['model_state_dict'])
    
    generated = generate_text(model, vocab_info, '$seed', max_length=50, temperature=0.8)
    print(f'Generated: {generated}')
except Exception as e:
    print(f'Error: {e}')
"

print("✅ TASK 3 COMPLETE: Text generation with seed phrases!")


In [ ]:
# ===============================================
# TASK 4: Model Performance Evaluation
# ===============================================
print("📊 TASK 4: Evaluating Model Performance...")

# Comprehensive model evaluation with metrics
!python -m src.evaluate --model_path outputs/best_model.pt --outdir outputs --num_samples 10 --max_length 100

# Display evaluation results
from IPython.display import Image, display
import pandas as pd
import os

print("📈 Model Performance Metrics:")
if os.path.exists('outputs/evaluation_metrics.csv'):
    metrics = pd.read_csv('outputs/evaluation_metrics.csv')
    display(metrics)

print("\n📊 Evaluation Visualizations:")
if os.path.exists('outputs/evaluation_results.png'):
    display(Image('outputs/evaluation_results.png'))

print("\n📝 Generated Text Samples:")
if os.path.exists('outputs/generated_texts.txt'):
    with open('outputs/generated_texts.txt', 'r') as f:
        generated_samples = f.read()
        print(generated_samples[:1000] + "..." if len(generated_samples) > 1000 else generated_samples)

print("✅ TASK 4 COMPLETE: Model performance evaluated with perplexity and accuracy!")


In [ ]:
# ===============================================
# TASK 5: Ablation Study - Model Component Analysis
# ===============================================
print("⚡ TASK 5: Running Ablation Study...")
print("Testing: Hidden Size, Number of Layers, Dropout, RNN Type")

# Run comprehensive ablation study (balanced epochs for comparison)
!python -m src.ablation_study --outdir outputs --epochs 12

# Display ablation results
import pandas as pd
from IPython.display import Image, display

print("📊 Ablation Study Results:")
if os.path.exists('outputs/ablation_results.csv'):
    ablation_results = pd.read_csv('outputs/ablation_results.csv')
    display(ablation_results)

print("🏆 Best Configuration Found:")
if os.path.exists('outputs/best_ablation_configs.json'):
    import json
    with open('outputs/best_ablation_configs.json', 'r') as f:
        best_configs = json.load(f)
    for component, config in best_configs.items():
        print(f"{component}: {config}")

print("📈 Ablation Study Visualization:")
if os.path.exists('outputs/ablation_study_results.png'):
    display(Image('outputs/ablation_study_results.png'))

print("✅ TASK 5 COMPLETE: Ablation study finished!")


In [ ]:
# ===============================================
# TASK 6: Optimal Model Training & Performance Comparison
# ===============================================
print("🏆 TASK 6: Training Optimal Model with Best Configuration...")

# Train model with optimal hyperparameters from ablation study
!python -m src.train_optimal --epochs 25 --best_config_file outputs/best_ablation_configs.json --outdir outputs

# Compare baseline vs optimal model performance
print("⚖️ Model Performance Comparison:")
if os.path.exists('outputs/model_comparison.csv'):
    comparison = pd.read_csv('outputs/model_comparison.csv')
    display(comparison)

print("📈 Optimal Model Training Curves:")
if os.path.exists('outputs/optimal_training_curves.png'):
    display(Image('outputs/optimal_training_curves.png'))

# Final evaluation with optimal model
!python -m src.evaluate --model_path outputs/optimal_model.pt --outdir outputs --num_samples 15 --max_length 100

print("✅ TASK 6 COMPLETE: Optimal model trained and compared!")


In [ ]:
# ===============================================
# SUMMARY & RESULTS OVERVIEW
# ===============================================
print("📋 FINAL RESULTS SUMMARY")
print("="*50)

# List all generated files
print("📁 Generated Files:")
!ls -la outputs/

# Show key metrics
import pandas as pd
import os

if os.path.exists('outputs/evaluation_metrics.csv'):
    print("\n🎯 Final Model Performance:")
    metrics = pd.read_csv('outputs/evaluation_metrics.csv')
    display(metrics)

if os.path.exists('outputs/model_comparison.csv'):
    print("\n⚖️ Baseline vs Optimal Model:")
    comparison = pd.read_csv('outputs/model_comparison.csv')
    display(comparison)

if os.path.exists('outputs/ablation_results.csv'):
    print("\n📊 Best Hyperparameters Found:")
    ablation = pd.read_csv('outputs/ablation_results.csv')
    best_row = ablation.loc[ablation['perplexity'].idxmin()]  # Lower perplexity is better
    print(f"Best Configuration: {best_row.to_dict()}")

print("\n🎭 Sample Generated Text:")
if os.path.exists('outputs/generated_texts.txt'):
    with open('outputs/generated_texts.txt', 'r') as f:
        sample_text = f.read()[:500]  # Show first 500 characters
        print(sample_text + "..." if len(sample_text) >= 500 else sample_text)

print("\n🎉 ALL Q2 TASKS COMPLETED SUCCESSFULLY!")
print("✅ Dataset loaded and preprocessed")
print("✅ RNN model implemented with custom embeddings")
print("✅ Model trained with monitoring curves")
print("✅ Text generation with seed phrases")
print("✅ Performance evaluation with perplexity/accuracy")
print("✅ Ablation study completed and compared")


In [ ]:
# ===============================================
# DOWNLOAD ALL RESULTS - Q2_OUTPUTS
# ===============================================
print("📦 Preparing Q2_OUTPUTS for download...")

# Show what files we generated
print("📁 Generated Files:")
!ls -la outputs/

# Create single comprehensive ZIP file
print("\n🗜️ Creating Q2_OUTPUTS.zip...")
!zip -r /content/Q2_OUTPUTS.zip outputs/ -x "*.pyc" "*__pycache__*"

# Download the complete package
from google.colab import files
print("\n⬇️ Downloading Q2_OUTPUTS.zip to your local CPU...")
files.download('/content/Q2_OUTPUTS.zip')

print("\n✅ DOWNLOAD COMPLETE!")
print("📦 File downloaded: Q2_OUTPUTS.zip")
print("💻 Check your Downloads folder")

print("\n📋 Your Q2_OUTPUTS.zip contains:")
print("🔹 All trained RNN models (.pt files)")
print("🔹 Training and validation curves")
print("🔹 Generated Shakespeare text samples")
print("🔹 Performance evaluation metrics")
print("🔹 Perplexity and accuracy analysis")
print("🔹 Complete ablation study results")
print("🔹 Model comparison data")
print("🔹 All generated analyses and plots")


In [ ]:
# ===============================================
# INTERACTIVE TEXT GENERATION (BONUS)
# ===============================================
print("🎭 BONUS: Interactive Shakespeare Text Generation")
print("Try generating text with custom seed phrases!")

import torch
from src.model import ShakespeareRNN
from src.utils import generate_text

try:
    # Load the optimal model
    checkpoint = torch.load('outputs/optimal_model.pt', map_location='cpu')
    vocab_info = checkpoint['vocab_info']
    model_info = checkpoint['model_info']

    # Create and load model
    model = ShakespeareRNN(
        vocab_size=model_info['vocab_size'],
        embedding_dim=model_info['embedding_dim'],
        hidden_size=model_info['hidden_size'],
        num_layers=model_info['num_layers'],
        dropout=0.0,
        rnn_type=model_info['rnn_type']
    )
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # Interactive generation function
    def generate_interactive(seed_text, max_length=50, temperature=1.0):
        generated = generate_text(
            model=model,
            vocab_info=vocab_info,
            seed_text=seed_text,
            max_length=max_length,
            temperature=temperature,
            device=torch.device('cpu')
        )
        return generated

    # Example generations with different temperatures
    print("\n🎯 Text Generation Examples with Different Creativity Levels:")
    
    seed = "To be or not to"
    temperatures = [0.5, 0.8, 1.2]
    temp_names = ["Conservative", "Balanced", "Creative"]
    
    for temp, name in zip(temperatures, temp_names):
        generated = generate_interactive(seed, max_length=60, temperature=temp)
        print(f"\n{name} (temp={temp}):")
        print(f"Seed: '{seed}'")
        print(f"Generated: '{generated}'")
    
    print("\n✨ Try your own seeds by modifying the code above!")
    print("🎛️ Adjust temperature: 0.5 (conservative) to 1.5 (very creative)")
    
except Exception as e:
    print(f"⚠️ Interactive generation not available: {e}")
    print("Make sure the optimal model has been trained successfully.")

print("\n🎉 Q2 Shakespeare RNN Project Complete!")
print("📖 Your model can now generate Shakespeare-style text!")
